# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

In [25]:
import  sqlite3
import csv

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [4]:
con = sqlite3.connect("Chinook_Sqlite.sqlite")
cur = con.cursor()

cur.execute("""
SELECT firstname, lastname
FROM Employee
WHERE country=?
""", ("Canada",))

print(cur.fetchmany(5))
cur.close()
con.close()

[('Andrew', 'Adams'), ('Nancy', 'Edwards'), ('Jane', 'Peacock'), ('Margaret', 'Park'), ('Steve', 'Johnson')]


2. Найти и вывести на экран названия всех альбомов группы Accept

In [7]:
con = sqlite3.connect("Chinook_Sqlite.sqlite")
cur = con.cursor()

cur.execute("""
SELECT title
FROM Album
WHERE artistid in (SELECT ArtistId from Artist WHERE name="Accept");
""")

print(cur.fetchmany(5))
cur.close()
con.close()

[('Balls to the Wall',), ('Restless and Wild',)]


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [8]:
con = sqlite3.connect("PI19-2.sqlite")
cur = con.cursor()
cur.execute("""
CREATE TABLE Student(id INTEGER, name VARCHAR(50))
""")
cur.execute("""
INSERT INTO Student(id, name) VALUES (1, "Liza")
""")
cur.execute("""
INSERT INTO Student(id, name) VALUES (2, "Alisa")
""")
cur.execute("""
INSERT INTO Student(id, name) VALUES (3, "Seva")
""")
con.commit()
print(cur.fetchmany(5))
cur.close()
con.close()

OperationalError: table Student already exists

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [47]:
con = sqlite3.connect("recipes.db")
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [48]:
cur.execute("""
CREATE TABLE Recipe(id INTEGER, name VARCHAR(50), minutes INTEGER,
     submitted DATE, description VARCHAR(350), n_ingredients INTEGER)
""")
con.commit()

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [49]:
cur.execute("""
CREATE TABLE Review(id INTEGER, user_id INTEGER, recipe_id INTEGER,
     date DATE, rating INTEGER, review VARCHAR(350),
     FOREIGN KEY (recipe_id) REFERENCES Recipe (id))
""")
con.commit()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [51]:
with open('reviews_sample.csv','r', encoding='utf-8') as f: 
    revs = csv.DictReader(f)
    next(revs)
    for row in revs:
        cur.execute("""INSERT INTO Review (id,user_id,recipe_id,date,rating,review) 
        VALUES (?, ?, ?, ?, ?, ?);""", (row[''], row['user_id'], row['recipe_id'], row['date'],
         row['rating'], row['review']))
con.commit()

In [52]:
with open('recipes_sample_with_tags_ingredients.csv','r', encoding='utf-8') as f: 
    revs = csv.DictReader(f)
    next(revs)
    for row in revs:
        cur.execute("""INSERT INTO Recipe (id,name,minutes,submitted,description,n_ingredients)
         VALUES (?, ?, ?, ?, ?, ?);""", (row['id'], row['name'], row['minutes'], row['submitted'],
         row['description'], row['n_ingredients']))
con.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [53]:
cur.execute("""
SELECT * 
FROM Recipe
WHERE n_ingredients = 10;
""")

print(cur.fetchmany(5))

[(503475, 'blepandekager   danish   apple pancakes', 50, '2013-07-08', "this recipe has been posted here for play in zwt9 - scandinavia.  this recipe was found at website: mindspring.com - christian's danish recipes.", 10), (125195, 'kelly s creamy cheddar pea salad', 20, '2005-06-09', "i'm not a big fan of peas, but like them in things. this salad is one of my favorites. happy to share!", 10), (120297, 'middle eastern   twice baked potatoes', 110, '2005-05-02', 'a delicious yet low-fat twice-baked potato side dish with a blend of classic middle eastern flavours: chickpeas, cumin and coriander.  serve with roasts or with your favourite middle eastern main course dish.  adapted from "practical cookery: low fat".  these potatoes could also be served on their own as a light vegetarian meal. in view of the comments by reviewers about the potatoes being a bit dry, i have added a tablespoon of tahini. i have also added some garlic. i\'m not sure how the recipe escaped having garlic in it whe

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [58]:
cur.execute("""
SELECT name
FROM Recipe
WHERE minutes = (SELECT max(minutes) FROM Recipe);
""")

print(cur.fetchall())

[('blueberry liqueur',), ('strawberry liqueur',)]


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [92]:
p = input()
cur.execute("""
    SELECT *
    FROM Recipe
    WHERE id = ?;
    """, (p,))
con.commit() 
ls = cur.fetchall()
if len(ls) == 0:
    print("Данных нет") 
else: 
    print(ls)


Данных нет


236281

8. Найдите кол-во отзывов с рейтингом 5.

In [94]:
cur.execute("""
SELECT count(id)
FROM Review 
WHERE rating = 5;
""")

print(cur.fetchall()[0])

(91360,)


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [95]:
cur.execute("""
SELECT count(DISTINCT id)
FROM Review 
WHERE rating >= 4;
""")

print(cur.fetchall()[0])

(112226,)


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [96]:
cur.execute("""
SELECT count(id)
FROM Recipe
WHERE submitted >= '2010-01-01' and submitted <= '2010-12-31'
AND minutes >=15;
""")

print(cur.fetchall()[0])

(1319,)


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [99]:
cur.execute("""
SELECT r.id, r.name, v.user_id, v.date, v.rating
FROM Recipe r JOIN Review v
ON r.id = v.recipe_id
WHERE r.n_ingredients >= 3
ORDER BY r.id;
""")

print(cur.fetchmany(5))

[(48, 'boston cream pie', 32421, '2002-03-15', 0), (48, 'boston cream pie', 68674, '2004-05-03', 2), (55, 'betty crocker s southwestern guacamole dip', 53959, '2006-01-12', 4), (55, 'betty crocker s southwestern guacamole dip', 165567, '2006-03-31', 5), (55, 'betty crocker s southwestern guacamole dip', 851190, '2010-05-23', 5)]


In [46]:
cur.close()
con.close()